# golive2

A continuation of the **golive** Jupyter notebook (keeping it stable).


The pre-masked speed 'vv_masked' is the observation of interest: How does it vary over
the course of several years? I can think of a couple 'color vis' approaches without much thought:


1. Empirical: For each location and across annual time bins (say 1/10th of a year) accumulate 
mean speed and std dev with the idea of 're-coloring' everything by std dev / mean or for a given 
time using speed / mean speed. (Is this 'frequentist'?)


2. Treat each location + time series as a vector and apply a clustering algorithm like k-means.
(Is this 'bayesian'?)


Accumulating these data is most easily done using the xarray 


: Do we generate a 2D array of lists
that accumulate from each dataset? Or do we go with time built-in as a third dimension?  Each 
addition is from a pixel from the 819 (easting) x 828 (northing) grid; and at a certain mean-day; 
and with a particular L8 time interval (16, 32, 48, 64, 80). 


accumulator(x, y) += (t, dt, vx, vy, vv, corr, del_corr) 


Some screening on ingest: using lgo_mask and an ad hoc thresold on maximum speed.


Post-hoc screening: vx, vy consistent with other observations 


In [1]:
# Some Python utility code
from pathlib import Path
home_d = str(Path.home()) + '/'
data_d = home_d + 'data/'             # A non-repository location for datasets of interest

def dirobj(obj): return [x for x in dir(obj) if not x.startswith('_')]

def lsal(path=''):
    import os
    return os.popen('ls -al ' + path).readlines()

def ShowGitHubImage(username, repo, folder, source, localpath, localname, width, height):
    global home_d
    import requests, shutil
    from PIL import Image
    outf = localpath + '/' + localname
    f = 'https://raw.githubusercontent.com/' + username + '/' + repo + '/master/' + folder + '/' + source
    a = requests.get(f, stream = True)
    if a.status_code == 200:
        with open(outf, 'wb') as f:
            a.raw.decode_content = True
            shutil.copyfileobj(a.raw, f)
    return Image.open(outf).resize((width,height),Image.ANTIALIAS)

def ShowLocalImage(path, filename, width, height):
    from PIL import Image
    f = path + '/' + filename 
    return Image.open(f).resize((width,height),Image.ANTIALIAS)

# Test either of the 'Show Image' functions
# ShowGitHubImage('robfatland', 'othermathclub', 'images/cellular', 'conus_textile_shell_2.png', home_d, 'ctextile.jpg', 450, 250)
# ShowLocalImage(home_d, 'ctextile.jpg', 450, 250)

In [2]:
# what we need to get stuff done
!pip install utm

%matplotlib inline
from matplotlib import pyplot as plt
import pandas as pd
import netCDF4
import xarray as xr
from xarray import Dataset
import numpy as np
from ftplib import FTP
import utm

You are using pip version 9.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [27]:
# file grabbing code... this takes a few minutes 
golive2_d = data_d + 'golive2/'

if False:
    ftp = FTP('dtn.rc.colorado.edu')
    ftp.login()
    ftp.cwd('work/nsidc0710/nsidc0710_landsat8_golive_ice_velocity_v1.1/p063_r018')
    a=[]
    ftp.dir(a.append)    # This is a rather tricky line of Python as it passes an append method as an argument
    d = [b.split()[8] for b in a if len(b.split()) == 9 and b.split()[8][-3:]=='.nc']
    for i in range(len(d)):
        ftp.retrbinary('RETR ' + d[i], open(golive2_d + d[i], 'wb').write)  
        print('obtained file', i)
    ftp.quit()
    print(len(d))
    # for c in d: print(c)

In [59]:
ds = xr.open_mfdataset(golive2_d + 'L8_063_018_016_2013_143_2013_159_v1.1.nc')
# ds1['image_pair_times']
# ds1['time'] = xr.Variable('time', ds1['image_pair_times'].attrs['mid_date'])
# ds1
print(type(ds['image_pair_times'].attrs['mid_date']))
print(ds['image_pair_times'].attrs['mid_date'])
s = ds['image_pair_times'].attrs['mid_date']
ts = pd.Timestamp(s)
ds['time'] = xr.Variable('time', [ts])
ds

<class 'str'>
2013-05-31T00:00:00


<xarray.Dataset>
Dimensions:                                         (chars: 4280, time: 1, x: 803, y: 786)
Coordinates:
  * x                                               (x) float64 3.373e+05 ...
  * y                                               (y) float64 6.779e+06 ...
  * chars                                           (chars) int32 0 1 2 3 4 ...
  * time                                            (time) datetime64[ns] 2013-05-31 ...
Data variables:
    image_pair_times                                |S1 ...
    input_image_details                             |S1 ...
    transverse_mercator                             |S1 ...
    offset_correction                               |S1 ...
    applied_bilinear_x_offset_correction_in_pixels  (y, x) float32 dask.array<shape=(786, 803), chunksize=(786, 803)>
    applied_bilinear_y_offset_correction_in_pixels  (y, x) float32 dask.array<shape=(786, 803), chunksize=(786, 803)>
    vx                                              (y, x) flo

In [60]:
#ds1
#ds1.squeeze('chars', drop=True)
#ds1['vv_masked'].sel(x=':', )
#del ds1['chars']
#ds1 = ds1.sel(x=slice(-1), y=slice(-1))
#ds1
# def pp(ds1):
#     del ds1['processing_log']
#     del ds1['chars']
#     ds1.rename({'chars':'band'},inplace=True)
#     return ds1.assign_coords(band=1)

def pp(ds):
    s = ds['image_pair_times'].attrs['mid_date']
    ts = pd.Timestamp(s)
    ds['time'] = xr.Variable('time', [ts])
    return ds

m = xr.open_mfdataset(golive2_d + '*.nc', preprocess = pp, decode_cf=False, autoclose=True, concat_dim='time')

In [8]:
ds2 = xr.open_mfdataset(golive2_d + 'L8_063_018_016_2013_159_2013_175_v1.1.nc')
ds2

<xarray.Dataset>
Dimensions:                                         (chars: 4280, x: 800, y: 786)
Coordinates:
  * x                                               (x) float64 3.388e+05 ...
  * y                                               (y) float64 6.779e+06 ...
  * chars                                           (chars) int32 0 1 2 3 4 ...
Data variables:
    image_pair_times                                |S1 ...
    input_image_details                             |S1 ...
    transverse_mercator                             |S1 ...
    offset_correction                               |S1 ...
    applied_bilinear_x_offset_correction_in_pixels  (y, x) float32 dask.array<shape=(786, 800), chunksize=(786, 800)>
    applied_bilinear_y_offset_correction_in_pixels  (y, x) float32 dask.array<shape=(786, 800), chunksize=(786, 800)>
    vx                                              (y, x) float32 dask.array<shape=(786, 800), chunksize=(786, 800)>
    vy                                    

In [28]:
m = xr.open_mfdataset(golive2_d + '*.nc', decode_cf=False, autoclose=True)

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [47]:
# print(m['image_pair_times'])
m['image_pair_times'].del_t, m['image_pair_times'].mid_time_decimal_year

('16.0', '2013.4109589')

In [48]:
vxm = m.vx_masked
vym = m.vy_masked
vvm = m.vv_masked
corr = m.corr
dcorr = m.del_corr

In [49]:
# This is Cargo Cult; but http://xarray.pydata.org/en/stable/examples/quick-overview.html#create-a-dataarray is a set
#   of helpful incremental examples

ds=Dataset(data_vars={'acc': '(y, x) float32 dask.array<shape=(828, 819), chunksize=(828, 819)>'}, \
           coords={'* x': 'x yeah', '* y': 'y yeah'}, attrs={'spongy': 'yes'}, compat='broadcast_equals')

In [50]:
ds

<xarray.Dataset>
Dimensions:  ()
Coordinates:
    * x      <U6 'x yeah'
    * y      <U6 'y yeah'
Data variables:
    acc      <U65 '(y, x) float32 dask.array<shape=(828, 819), chunksize=(828, 819)>' ...
Attributes:
    spongy:   yes